# 2_네이버 뉴스  
# (3) lda 토픽 모델링

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import collections
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

In [5]:
naver_news = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/preprocessed_naver_news.csv")

In [6]:
naver_news = naver_news[['month', 'title', 'token_lst', 'token']]

In [7]:
naver_news

,month,title,token_lst,token
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 위 오른 국산 전기차는,"['볼보', '폴스타', '안전', '자동차', '위', '국산', '전기차']",볼보 폴스타 안전 자동차 위 국산 전기차
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,"['현대제철', '친환경', '전기차', '시장', '공략', '수익', '회복',...",현대제철 친환경 전기차 시장 공략 수익 회복 전
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"['테슬라', '내리', '포드', '세일', '전기', '차', '치킨', '게임...",테슬라 내리 포드 세일 전기 차 치킨 게임 시작
3,2023-01,SK온 현대차 아이오닉 배터리 서산에서 만든다,"['SK', '현대차', '아이오닉', '배터리', '서산']",SK 현대차 아이오닉 배터리 서산
4,2023-01,공격적 수주 현대모비스 해외 일감 배 껑충 사상 최대,"['공격', '수주', '현대모비스', '해외', '일감', '배', '껑충', '...",공격 수주 현대모비스 해외 일감 배 껑충 사상 최대
...,...,...,...,...
2295,2024-11,현대차 말레이시아에 억 투자 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산
2296,2024-11,현대차 말레이시아에 천억 투자해 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산
2297,2024-11,현대차 말레이시아에 억원 투자 만대 생산체제 구축,"['현대차', '말레이시아', '투자', '생산', '체제', '구축']",현대차 말레이시아 투자 생산 체제 구축
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,"['현대모비스', '내년', 'CES', '참가', '글로벌', '자동차', '고객...",현대모비스 내년 CES 참가 글로벌 자동차 고객 확보


In [ ]:
jan23 = naver_news[naver_news['month']=='2023-01']
feb23 = naver_news[naver_news['month']=='2023-02']
mar23 = naver_news[naver_news['month']=='2023-03']
apr23 = naver_news[naver_news['month']=='2023-04']
may23 = naver_news[naver_news['month']=='2023-05']
jun23 = naver_news[naver_news['month']=='2023-06']
jul23 = naver_news[naver_news['month']=='2023-07']
aug23 = naver_news[naver_news['month']=='2023-08']
sep23 = naver_news[naver_news['month']=='2023-09']
oct23 = naver_news[naver_news['month']=='2023-10']
nov23 = naver_news[naver_news['month']=='2023-11']
dec23 = naver_news[naver_news['month']=='2023-12']

In [ ]:
jan24 = naver_news[naver_news['month']=='2024-01']
feb24 = naver_news[naver_news['month']=='2024-02']
mar24 = naver_news[naver_news['month']=='2024-03']
apr24 = naver_news[naver_news['month']=='2024-04']
may24 = naver_news[naver_news['month']=='2024-05']
jun24 = naver_news[naver_news['month']=='2024-06']
jul24 = naver_news[naver_news['month']=='2024-07']
aug24 = naver_news[naver_news['month']=='2024-08']
sep24 = naver_news[naver_news['month']=='2024-09']
oct24 = naver_news[naver_news['month']=='2024-10']
nov24 = naver_news[naver_news['month']=='2024-11']

[과정]  
0. 텍스트 데이터 결합
1. Count 기반 벡터화
2. 최적의 topic 개수 탐색
3. LDA 토픽 모델링
4. LDA 시각화

## 0. 월별 텍스트 데이터 결합

In [23]:
import ast

In [24]:
naver_news['token_lst'] = naver_news['token_lst'].apply(ast.literal_eval)

In [25]:
month_tokens_list = (naver_news.groupby('month')['token_lst'].apply(list).tolist())

In [26]:
print(len(month_tokens_list))

23


In [27]:
print(len(month_tokens_list[0]))

100


In [28]:
print(month_tokens_list[0][:5])

[['볼보', '폴스타', '안전', '자동차', '위', '국산', '전기차'], ['현대제철', '친환경', '전기차', '시장', '공략', '수익', '회복', '전'], ['테슬라', '내리', '포드', '세일', '전기', '차', '치킨', '게임', '시작'], ['SK', '현대차', '아이오닉', '배터리', '서산'], ['공격', '수주', '현대모비스', '해외', '일감', '배', '껑충', '사상', '최대']]


## 1. Count 기반 벡터화

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
# LDA는 Count 기반의 Vectorizer만 적용
count_vectorizer = CountVectorizer(max_features=1000, min_df=3, ngram_range=(1,3))
## max_features : 포함시킬 최대 단어 수, 빈도 순으로 최대 1000단어까지만 포함시킴
## min_df : 단어장에 포함되기 위한 최소 빈도, 3개 문서 미만으로 등장하는 단어는 제외
## bigram, trigram까지 포함 -> 1-gram ~ 3-gram

In [31]:
# 전체 데이터를 하나의 리스트로 결합하여 단어 집합을 학습
all_tokens = naver_news['token'].tolist()
count_vectorizer.fit(all_tokens)

CountVectorizer(max_features=1000, min_df=3, ngram_range=(1, 3))

In [32]:
# 월별로 transform을 적용하여 벡터화
month_vectors = [count_vectorizer.transform([text]) for text in naver_news.groupby('month')['token'].apply(' '.join)]

In [35]:
for i, vect in enumerate(month_vectors):
    print((f"{2023+(i//12)}/{i%12+1} - Shape of vectorized data: {vect.shape}"))

2023/1 - Shape of vectorized data: (1, 1000)
2023/2 - Shape of vectorized data: (1, 1000)
2023/3 - Shape of vectorized data: (1, 1000)
2023/4 - Shape of vectorized data: (1, 1000)
2023/5 - Shape of vectorized data: (1, 1000)
2023/6 - Shape of vectorized data: (1, 1000)
2023/7 - Shape of vectorized data: (1, 1000)
2023/8 - Shape of vectorized data: (1, 1000)
2023/9 - Shape of vectorized data: (1, 1000)
2023/10 - Shape of vectorized data: (1, 1000)
2023/11 - Shape of vectorized data: (1, 1000)
2023/12 - Shape of vectorized data: (1, 1000)
2024/1 - Shape of vectorized data: (1, 1000)
2024/2 - Shape of vectorized data: (1, 1000)
2024/3 - Shape of vectorized data: (1, 1000)
2024/4 - Shape of vectorized data: (1, 1000)
2024/5 - Shape of vectorized data: (1, 1000)
2024/6 - Shape of vectorized data: (1, 1000)
2024/7 - Shape of vectorized data: (1, 1000)
2024/8 - Shape of vectorized data: (1, 1000)
2024/9 - Shape of vectorized data: (1, 1000)
2024/10 - Shape of vectorized data: (1, 1000)
2024/1

In [37]:
print(month_vectors[0])

  (0, 23)	6
  (0, 24)	1
  (0, 26)	3
  (0, 30)	1
  (0, 33)	2
  (0, 34)	1
  (0, 35)	5
  (0, 38)	2
  (0, 42)	5
  (0, 45)	1
  (0, 48)	1
  (0, 50)	1
  (0, 51)	1
  (0, 54)	2
  (0, 56)	1
  (0, 66)	2
  (0, 68)	3
  (0, 72)	1
  (0, 75)	1
  (0, 79)	1
  (0, 82)	1
  (0, 83)	1
  (0, 88)	2
  (0, 89)	3
  (0, 92)	1
  :	:
  (0, 875)	1
  (0, 883)	13
  (0, 885)	1
  (0, 886)	4
  (0, 892)	1
  (0, 893)	1
  (0, 898)	2
  (0, 899)	1
  (0, 906)	5
  (0, 907)	5
  (0, 912)	51
  (0, 916)	13
  (0, 917)	1
  (0, 922)	2
  (0, 925)	11
  (0, 930)	5
  (0, 942)	2
  (0, 950)	1
  (0, 954)	5
  (0, 963)	1
  (0, 975)	2
  (0, 981)	2
  (0, 991)	2
  (0, 993)	2
  (0, 998)	1


## 2. LDA 토픽 모델링

In [47]:
from sklearn.decomposition import LatentDirichletAllocation

### way1) scikitlearn LDA

In [61]:
for i in range(len(month_vectors)):
  lda = LatentDirichletAllocation(n_components=3)
  lda.fit(month_vectors[i].toarray())

  feature_names = count_vectorizer.get_feature_names_out()
  print(f"Top 3 Topics for {2023 + (i // 12)}/{i % 12 + 1}")
  for topic_idx, topic in enumerate(lda.components_):
    print(f"  Topic #{topic_idx+1}")
    print("   " + " ".join(feature_names[i] for i in topic.argsort()[:-11:-1]))
  print()

Top 3 Topics for 2023/1
  Topic #1
   현대차 전기차 그룹 시장 현대 현대차 그룹 전기 올해 실적 기아
  Topic #2
   수주 품질 영업 이익 그룹 미국 퍼스트 오토 오토 에버 분석 글로벌 판매 전기차 부품
  Topic #3
   수주 품질 영업 이익 그룹 미국 퍼스트 오토 오토 에버 분석 글로벌 판매 전기차 부품

Top 3 Topics for 2023/2
  Topic #1
   전기차 현대차 기아 아이오닉 전기 미국 현대 그룹 테슬라 현대차 기아
  Topic #2
   조사 드라이브 전기차 수소 디자인 공동 해외 수주 전기차 전용 전기차 전용 공장 해외 전기차 출시
  Topic #3
   조사 드라이브 전기차 수소 디자인 공동 해외 수주 전기차 전용 전기차 전용 공장 해외 전기차 출시

Top 3 Topics for 2023/3
  Topic #1
   에너지 점유 현대 뱅크 리콜 현대 기아 전망 전시 르노 전환 로보
  Topic #2
   에너지 점유 현대 뱅크 리콜 현대 기아 전망 전시 르노 전환 로보
  Topic #3
   전기차 현대차 모빌리티 서울 서울 모빌리티 전기 ev 공개 기아 충전

Top 3 Topics for 2023/4
  Topic #1
   현대차 전기차 기아 미국 현대차 기아 현대 영업 분기 배터리 보조금
  Topic #2
   대세 전기차 핵심 내년 직원 중국 전기차 중고 주주 다시 주목 주가
  Topic #3
   대세 전기차 핵심 내년 직원 중국 전기차 중고 주주 다시 주목 주가

Top 3 Topics for 2023/5
  Topic #1
   현대차 배터리 lg 전기차 공장 기아 그룹 현대차 그룹 미국 합작
  Topic #2
   단독 거리 충전기 급증 기계 기관 기술 추가 체결 차종
  Topic #3
   단독 거리 충전기 급증 기계 기관 기술 추가 체결 차종

Top 3 Topics for 2023/6
  Topic #1
   현대차 전기차 전기 기아 충전 현대 

### way2) Gensim LDA

In [100]:
for i in range(len(month_tokens_list)):
    tokenized_documents = month_tokens_list[i]
    dictionary = Dictionary(tokenized_documents)
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=3,
        random_state=0,
        iterations=100,
        passes=10
    )

    print(f"Top 3 Topics for {2023 + (i // 12)}/{i % 12 + 1}")
    for idx, topic in lda_model.print_topics(num_words=10):
        # 토픽에서 단어만 추출하여 출력
        words = [word.split("*")[1].strip('"') for word in topic.split(" + ")]
        print(f"  Topic #{idx+1}: {' '.join(words)}")
    print()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Top 3 Topics for 2023/1
  Topic #1: 전기차 현대차 시장 미국 테슬라 기아 현대 올해 자동차 IRA
  Topic #2: 현대차 전기차 차 그룹 현대 전년 미래 매출 위 테슬라
  Topic #3: 현대차 판매 차 전기 실적 돌파 최대 올해 누적 사상

Top 3 Topics for 2023/2
  Topic #1: 전기차 현대차 차 기아 위 EV 생산 아이오닉 전기 충전
  Topic #2: 전기차 현대차 차 그룹 전기 미국 올해 자동차 현대 부품
  Topic #3: 현대차 전기차 가격 더 기아 테슬라 아이오닉 차 현대 충전

Top 3 Topics for 2023/3
  Topic #1: 현대차 전기차 그룹 공개 현대 충전 부산 엑스포 개발 유치
  Topic #2: 모빌리티 전기차 쇼 서울 전기 차 EV 기아 공개 대형
  Topic #3: 전기차 현대차 사장 중국 충전 SK 파워 로봇 기술 장재훈

Top 3 Topics for 2023/4
  Topic #1: 현대차 토요타 영업 그룹 기아 모빌리티 현대모비스 분기 차 한국
  Topic #2: 현대차 전기차 미국 기아 영업 분기 보조금 SK 조 배터리
  Topic #3: 전기차 현대 현대차 배터리 차 공급 질주 보조금 전기 부품

Top 3 Topics for 2023/5
  Topic #1: 현대차 기아 이피 대유 수주 휠 스티어링 전기차 자동차 판매
  Topic #2: 현대차 LG 솔 배터리 미국 합작 공장 그룹 북미 법인
  Topic #3: 전기차 현대차 배터리 현대모비스 자동차 시장 생산 전기 기아 규모

Top 3 Topics for 2023/6
  Topic #1: 차 현대차 전기 아이오닉 충전 그룹 선정 올해 현대 드라이버
  Topic #2: 전기차 현대차 차 전기 투자 기아 그룹 배터리 GV 충전
  Topic #3: 현대차 전기차 기아 테슬라 현대 반도체 미국 차 일본 속도

Top 3 Topics for 2023/7
  Topic #1: 현대차 전

## 3. LDA 시각화

In [62]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.2 MB/s eta 0:00:00


In [93]:
import pyLDAvis.gensim_models
from scipy.sparse import csr_matrix
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import os

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
def monthly_lda_viz(month_tokens_list, month_vectors, count_vectorizer):
    for i in range(len(month_tokens_list)):
      tokenized_documents = month_tokens_list[i]
      flattend_tokens = [token for doc in tokenized_documents for token in doc]

      dictionary = Dictionary(tokenized_documents)
      corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

      lda = LdaModel(corpus = corpus, id2word = dictionary, num_topics=3, iterations=100, passes=10, random_state=0)

      print(f"Visualizing Topics for {2023 + (i // 12)}/{i % 12 + 1}")

      viz = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)

      output_dir = "/content/gdrive/MyDrive/통계분석실습/기말프로젝트"
      output_path = os.path.join(output_dir, f"lda_viz_{23 + (i // 12)}_{i % 12 + 1}.html")
      pyLDAvis.save_html(viz, output_path)
      print(f"Visualization for {2023 + (i // 12)}/{i % 12 + 1} saved as HTML.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
monthly_lda_viz(month_tokens_list, month_vectors, count_vectorizer)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Visualizing Topics for 2023/1
Visualization for 2023/1 saved as HTML.
Visualizing Topics for 2023/2
Visualization for 2023/2 saved as HTML.
Visualizing Topics for 2023/3
Visualization for 2023/3 saved as HTML.
Visualizing Topics for 2023/4
Visualization for 2023/4 saved as HTML.
Visualizing Topics for 2023/5
Visualization for 2023/5 saved as HTML.
Visualizing Topics for 2023/6
Visualization for 2023/6 saved as HTML.
Visualizing Topics for 2023/7
Visualization for 2023/7 saved as HTML.
Visualizing Topics for 2023/8
Visualization for 2023/8 saved as HTML.
Visualizing Topics for 2023/9
Visualization for 2023/9 saved as HTML.
Visualizing Topics for 2023/10
Visualization for 2023/10 saved as HTML.
Visualizing Topics for 2023/11
Visualization for 2023/11 saved as HTML.
Visualizing Topics for 2023/12
Visualization for 2023/12 saved as HTML.
Visualizing Topics for 2024/1
Visualization for 2024/1 saved as HTML.
Visualizing Topics for 2024/2
Visualization for 2024/2 saved as HTML.
Visualizing To